# <span style="color:gold">**Análisis exploratorio de datos en un Modelo de Bloques (Parte 1)**</span>
***

### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
***

### **Descripción**

En este notebook, desarrollaremos un flujo de trabajo para el análisis exploratorio de datos usando un <span style="color:gold">modelo de bloques</span> dentro de Python.

Este Notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología.
***

## <span style="color:lightgreen">**Objetivos de un Análisis Exploratorio de Datos (EDA)**</span>

- **Maximizar** el entendimiento que se tiene acerca del dataset a través de resúmenes estadísticos y representaciones visuales.
- **Descubrir** tendencias y patrones en los datos a través del análisis univariable, bivariable y multivariable.
- **Limpiar** el dataset, reemplazando o eliminando aquellos valores anómalos o incorrectos. 
- **Generar** preguntas e hipótesis que podrán ser probadas luego usando otros métodos estadísticos.

<br>

# **1. Tratamiento de datos**
## **1.1. Cargando información en Python**

Empezaremos importando las siguientes librerías:
- `numpy`: para operar datos en filas y columnas (matrices)
- `pandas`: para procesar el modelo de bloques
- `matplotlib`: para generar gráficos en 2D del modelo
- `seaborn`: para agregar un estilo de visualización más agradable
- `ipywidgets`: para crear funciones y gráficos interactivos

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", context="talk")
import ipywidgets as widgets

# Número de decimales a mostrar en un DataFrame
pd.set_option("display.float_format", lambda x: f"{x:.5f}")

Usaremos la función `read_csv` para extraer la información del archivo `block_model.csv` y lo asignaremos a una variable llamada `data`:
> De antemano, debemos saber que el modelo de bloques corresponde a un yacimiento de tipo pórfido de Cu-Mo.

In [ ]:
data = pd.read_csv("block_model.csv")

Ahora, utilizaremos el método `head` para observar las primeras 5 filas de la tabla:

In [ ]:
data.head()

## **1.2. Modificación de valores dentro de un DataFrame**

### **Modificación del Índice**
Tenemos una columna llamada `ID` que parece corresponder al índice de la tabla.\
Podemos eliminar esta columna o establecerla como índice usando el método `set_index`:
> El parámetro `inplace=True` es usado para guardar los cambios dentro de la misma variable.

In [ ]:
data.set_index("ID", inplace=True)

Observamos el resultado:

In [ ]:
data.head()

### **Tipos de datos en un DataFrame**
Ahora que ya establecimos un índice, usaremos el atributo `dtypes` para observar el tipo de dato asignado a cada columna de la tabla:

In [ ]:
data.dtypes

En resumen, podemos notar las siguientes características dentro de la tabla:
- Cada fila representa uno de los bloques del modelo
- `X`, `Y`, `Z`: coordenadas del centroide para cada bloque, representado en Python como `float`
- `CU (PCT)`: concentración de Cu en porcentaje, representado en Python como `string`
- `MO (PPM)`: concentración de Mo en ppm, representado en Python como `string`
- `LITOLOGIA`: tipo de roca asignada al bloque, representado en Python como `string`

### **Modificación de tipos de datos, reemplazo de nombres en columnas y valores vacíos**
- Empezaremos abreviando los nombre de las columnas `LITOLOGIA` por `LITO`, `CU (PCT)` por `CU_pct` y `MO (PPM)` por `MO_ppm`.
- Los valores "blank" en el modelo representan bloques vacíos, serán reemplazados por valores vacíos de tipo `numpy.nan`.
- Las columnas de Cu y Mo representan valores numéricos en `string`, transformaremos el tipo de dato de estas columnas a `float` para facilitar su procesamiento.

Para modificar el nombre de las columnas, usaremos el método `rename`:

In [ ]:
# Modificando el nombre de las columnas
data.rename(columns={"LITOLOGIA": "LITO", "CU (PCT)": "CU_pct", "MO (PPM)": "MO_ppm"}, inplace=True)

El método `replace` nos permite reemplazar valores dentro de un DataFrame:

In [ ]:
# Reemplazando valores vacíos "blank" por np.nan
data.replace("blank", np.nan, inplace=True)

El método `astype` es usado para modificar el tipo de dato usado en una o varias columnas:

In [ ]:
# Cambiando el tipo de dato en las columnas de Cu y Mo
data[["CU_pct", "MO_ppm"]] = data[["CU_pct", "MO_ppm"]].astype(float)

Ahora usaremos el método `sample` para observar filas aleatorias en la tabla:

In [ ]:
data.sample(6)

### **Eliminación de valores vacíos**
El método `isna` es usado para verificar la presencia de valores vacíos dentro de un DataFrame.\
Si la celda contiene un valor vacío de tipo `nan`, el resultado será `True`.\
Por otro lado, si la celda contiene información, el resultado será `False`.

In [ ]:
data.isna()

Si usamos la columna `LITO`, obtenemos un arreglo de valores lógicos:

In [ ]:
data["LITO"].isna()

Notamos que existen valores vacíos en las columnas de Cu, Mo y Litología.\
Calcularemos el número total de bloques vacíos y el porcentaje de estos respecto al total:

In [ ]:
# ¿Cuántos bloques contiene el modelo?
bloques = len(data)
# ¿Cuántos valores nulos existen?
nulo = sum(data["LITO"].isna())

# Resultados
print(f"Existen {bloques:,} bloques en el modelo, de los cuales {nulo:,} contienen valores nulos")
print(f"Los bloques con valores nulos representan el {nulo/bloques:.2%} del modelo")

Los bloques vacíos no nos interesan, por lo tanto, crearemos una copia del dataset original y trabajaremos con aquellos bloques que contengan información del modelo.\
Esto acelerará el procesamiento de los datos, debido a que solo trabajaremos usando el 50% del modelo original.\
Para remover las filas con valores vacíos usaremos el método `dropna` y crearemos una copia usando el método `copy`:

In [ ]:
modelo = data.dropna().copy()

In [ ]:
modelo.head()

Podemos reindexar las filas usando el método `reset_index`.
> El parámetro `drop=True` elimina el índice original.

In [ ]:
modelo.reset_index(drop=True, inplace=True)

In [ ]:
modelo.head()

# **2. Análisis univariable**
### **Resumen del DataFrame**
Podemos calcular el número total de filas en un DataFrame con la función `len`:

In [ ]:
print(f"El modelo tiene en total {len(modelo):,} bloques con información.")

El método `describe` permite visualizar un resumen estadístico de cada columna que posea datos numéricos de tipo `float` o `integer`.\
Si deseamos visualizar datos categóricos en `string`, utilizaremos la opción `include="all"`:

In [ ]:
modelo.describe(include="all")

El método `info` devuelve información acerca del índice, las columnas y la cantidad de memoria utilizada por el DataFrame:

In [ ]:
modelo.info()

## **2.1. Análisis de valores numéricos**
Empezaremos analizando los valores de Cu y Mo.\
Primero, asignaremos las columnas de Cu y Mo en las variables `cu` y `mo`.\
También guardaremos los valores mínimos y máximos de Cu y Mo en variables:

In [ ]:
cu = modelo["CU_pct"]
mo = modelo["MO_ppm"]

cu_min, cu_max = cu.min(), cu.max()
mo_min, mo_max = mo.min(), mo.max()

Y crearemos histogramas para las leyes de Cu y Mo:
> El parámetro `sharey=True` comparte el eje Y entre figuras de una misma fila.\
> El parámetro `histtype="step"` reemplaza las barras originales del histograma por una línea escalonada.\
> El parámero `density=True` convierte la frecuencia en una distribución de probabilidad.

In [ ]:
# Figura principal
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# Histograma de Cu y Mo
ax1.hist(x=cu, bins=50, log=True, histtype="step", fill=True, alpha=0.8)
ax2.hist(x=mo, bins=50, log=True, histtype="step", fill=True, alpha=0.8)

# Títulos
ax1.set_title("Cu (%)", fontsize=20)
ax2.set_title("Mo (ppm)", fontsize=20)

# Límites
ax1.set_xlim(cu_min, cu_max)
ax2.set_xlim(mo_min, mo_max)

# Grillas
ax1.grid()
ax2.grid()

plt.tight_layout()

Notamos que los valores de Cu y Mo en el modelo tienen una **distribución logarítmica**, es decir, la mayoría de bloques en el modelo presentan leyes bajas.

Ahora, importaremos la función `ECDF` de la librería `statsmodels` para generar funciones de distribución cumulativa (CDF) para el Cu y Mo:

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
cu_cdf = ECDF(cu)
mo_cdf = ECDF(mo)

Estas funciones nos permiten determinar la probabilidad de que la ley de Cu/Mo en un bloque se encuentre por debajo de cierto valor:

In [ ]:
print(f"El {cu_cdf(1):.1%} de los bloques tienen valores de Cu por debajo de 1.0%")
print(f"El {mo_cdf(200):.1%} de los bloques tienen valores de Mo por debajo de 200 ppm")

También podemos usar la función de `numpy` llamada `quantile` para obtener la ley de Cu/Mo asociado a cada cuantil (e.g. 90% es cuantil 0.9):

In [ ]:
cu_90 = np.quantile(cu, 0.9)
mo_90 = np.quantile(mo, 0.9)

print(f"El 90% de los bloques tienen valores de Cu por debajo de {cu_90:.2f} %")
print(f"El 90% de los bloques tienen valores de Mo por debajo de {mo_90:.0f} ppm")

Ahora, graficaremos los histogramas cumulativos para el Cu y Mo:
> El parámetro `cumulative=True` convierte el histograma en cumulativo.

In [ ]:
# Figura principal
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# Histograma cumulativo de Cu y Mo
ax1.hist(x=cu, bins=50, cumulative=True, log=True, histtype="step", density=True, fill=True, alpha=0.8)
ax2.hist(x=mo, bins=50, cumulative=True, log=True, histtype="step", density=True, fill=True, alpha=0.8)

# Títulos
ax1.set_title("Cu (%)", fontsize=20)
ax2.set_title("Mo (ppm)", fontsize=20)

# Límites
ax1.set_xlim(cu_min, cu_max)
ax2.set_xlim(mo_min, mo_max)

# Grillas
ax1.grid()
ax2.grid()

plt.tight_layout()

## **2.2. Análisis de valores categóricos**
Los datos contenidos en la columna `LITO` corresponden a los tipos de rocas asignados a cada bloque.\
Para observar cada valor único presente en esta columna usaremos el método `unique`:

In [ ]:
print(modelo["LITO"].unique())

Usando esta información, calcularemos el número de bloques asignados a cada tipo de roca.\
Usaremos el método `groupby` para ordenar el cuadro de acuerdo a cada tipo de roca en la columna `LITO`.\
Luego, aplicaremos el método `count` para obtener el total de bloques por cada tipo de roca:

In [ ]:
modelo.groupby("LITO")["LITO"].count()

Usando este arreglo, podemos generar un resumen de la litología:

In [ ]:
# Extraemos el conteo total en un diccionario, ordenado de menor a mayor
total = dict(modelo.groupby("LITO")["LITO"].count().sort_values())

rocas = list(total.keys())    # Tipo de roca
conteo = list(total.values()) # Número de bloques por tipo de roca

# Bucle para mostrar la cantidad y porcentaje de bloques por cada tipo de roca
for cont, roca in zip(conteo, rocas):
    print(f"Existen {cont:,} bloques ({cont/sum(conteo):.1%}) de tipo {roca}")

# Número total de bloques
print(f"De un total de {sum(conteo):,} bloques.")

Podemos volver a crear histogramas de Cu y Mo pero esta vez los dividiremos de acuerdo al tipo de litología:

In [ ]:
# Definimos una función que grafique histogramas para el Cu y Mo de acuerdo a un tipo de litología
def leyes_litologia(lito, cumulative=False):
    # Selecciona las leyes de Cu y Mo de aquellos bloques que pertenezcan a la litología
    cu_lito = cu[modelo["LITO"] == lito]
    mo_lito = mo[modelo["LITO"] == lito]
    
    # Crea la figura
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
        
    if cumulative:
        # Histograma cumulativo de Cu y Mo
        ax1.hist(x=cu_lito, bins=50, cumulative=True, log=True, histtype="step", density=True, fill=True, alpha=0.8)
        ax2.hist(x=mo_lito, bins=50, cumulative=True, log=True, histtype="step", density=True, fill=True, alpha=0.8)
        fig.suptitle(f"Distribuciones cumulativas de Cu y Mo - {lito}", fontsize=30)
    else:
        # Histograma de Cu y Mo
        ax1.hist(x=cu_lito, bins=50, log=True, histtype="step", fill=True, alpha=0.8)
        ax2.hist(x=mo_lito, bins=50, log=True, histtype="step", fill=True, alpha=0.8)
        fig.suptitle(f"Distribuciones de Cu y Mo - {lito}", fontsize=30)
    
    # Títulos
    ax1.set_title("Cu (%)", fontsize=20)
    ax2.set_title("Mo (ppm)", fontsize=20)
    
    # Límites
    ax1.set_xlim(cu_min, cu_max)
    ax2.set_xlim(mo_min, mo_max)

    # Grillas
    ax1.grid()
    ax2.grid()

    # Muestra la figura
    plt.tight_layout()
    
widgets.interact(leyes_litologia, lito=rocas, cumulative=False);

Usando esta información, vamos a clasificar categóricamente los bloques de acuerdo a su contenido de Cu y Mo.

# **3. Análisis Bivariable**
Ahora que ya hemos analizado las variables `CU_pct`, `MO_ppm` y `LITO` de manera individual, pasaremos a analizarlas por pares.\
Empezaremos con el par Cu-Mo, creando un diagrama de dispersión general:
> Dentro del parámetro `subplot_kw`, el valor `aspect` es usado para establecer una proporción específica entre los ejes de la figura (e.g. 1/1, 1/2, etc.).

In [ ]:
# Figura inicial
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"aspect":1/200})

# Diagrama de dispersión
ax.scatter(cu, mo, s=0.5)

# Colocamos etiquetas a los ejes X e Y
ax.set_xlabel("Cu (%)")
ax.set_ylabel("Mo (ppm)")

# Colocamos un título
ax.set_title("Correlación entre leyes de Cu y Mo", fontsize=25)

# Colocamos una grilla
ax.grid()

# Mostramos la figura
plt.show()

La figura anterior muestra la correlación entre las leyes de Cu y Mo de todos los bloques sin diferenciar el tipo de litología.\
Esto no nos dice mucho, así que visualizaremos la correlación de acuerdo a la litología utilizando un cuadro de varias figuras:
> El parámetro `sharex=True` comparte el eje X entre figuras de una misma columna.

In [ ]:
# Figura inicial (2 filas, 3 columnas)
fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(14, 10))

# Bucle para generar una figura de dispersión por cada tipo de roca
for ax, roca in zip(axs.flatten(), rocas):
    cu_roca = cu[modelo["LITO"] == roca]
    mo_roca = mo[modelo["LITO"] == roca]
  
    ax.scatter(cu_roca, mo_roca, s=0.5)
    ax.set_xticks([0, 1, 2, 3, 4, 5, 6])
    
    ax.set_xlabel("Cu (%)")
    ax.set_ylabel("Mo (ppm)")
    ax.set_title(roca)
    ax.grid()

# Colocamos un título para la figura
fig.suptitle("Correlación entre leyes de Cu y Mo por tipo de roca", fontsize=28)

# Mostramos la figura
plt.tight_layout()

Usaremos los siguientes intervalos para la clasificación de leyes de Cu y Mo:\
`CU_pct`: valores menores a 1 %, entre 1 a 2 %, y mayores a 2 %\
`MO_ppm`: valores menores a 200 ppm, entre 200 y 400 ppm, y mayores a 400 ppm.

Las siguientes funciones serán usadas para clasificar los valores de Cu y Mo:

In [ ]:
def intervalo_cu(row):
    if row < 1.0:
        return "Cu < 1.0 %"
    elif 1.0 <= row < 2.0:
        return "1.0 <= Cu < 2.0 %"
    elif row >= 2.0:
        return "Cu >= 2.0 %"
    
def intervalo_mo(row):
    if row < 200:
        return "Mo < 200 ppm"
    elif 200 <= row < 400:
        return "200 <= Mo < 400 ppm"
    elif row >= 400:
        return "Mo >= 400 ppm"

Usaremos el método `apply` para generar una nueva columna a partir de la columna de ley:

In [ ]:
modelo["CU"] = cu.apply(intervalo_cu)
modelo["MO"] = mo.apply(intervalo_mo)

Observamos el resultado:

In [ ]:
modelo.sample(5)

Ahora, generaremos un resumen de la clasificación:

In [ ]:
cu_leyes = dict(modelo.groupby("CU")["LITO"].count().sort_values())

intervalos = list(cu_leyes.keys())
conteo = list(cu_leyes.values())
total = sum(conteo)

for inter, cont in zip(intervalos, conteo):
    print(f"Existen {cont:,} ({cont/total:.1%}) bloques con ley: {inter}")

In [ ]:
mo_leyes = dict(modelo.groupby("MO")["LITO"].count().sort_values())

intervalos = list(mo_leyes.keys())
conteo = list(mo_leyes.values())
total = sum(conteo)

for inter, cont in zip(intervalos, conteo):
    print(f"Existen {cont:,} ({cont/total:.1%}) bloques con ley: {inter}")

Ya tenemos un mejor entendimiento sobre la distribución de leyes de Cu-Mo y su relación con respecto a la litología en el modelo.\
El siguiente paso es analizar esta información de manera espacial, haciendo uso de herramientas de visualización 2D y 3D.

Para terminar esta parte del tutorial, guardaremos la información en un nuevo archivo CSV llamado `modelo.csv`.

In [ ]:
modelo.to_csv("files/modelo.csv", index=False)

***